In [1]:
pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  pinecone-client pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 7.8 MB/s eta 0:00:00


In [3]:
from pinecone import Pinecone
import os

In [4]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

In [5]:
os.environ.get('PINECONE_API_KEY') 

'3bd6ebfe-51bc-479b-b529-6bdcf0a37ff5'

In [7]:
pc.list_indexes()

MaxRetryError: HTTPSConnectionPool(host='api.pinecone.io', port=443): Max retries exceeded with url: /indexes (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))

In [ ]:
# List indexes
indexes = pc.list_indexes()
print(indexes[0].name)

In [ ]:
# Describe index
pc.describe_index(indexes[0].name)

In [ ]:
#create index
new_index_name = "prices"
dimension = 64
metric = "cosine"

In [ ]:
# if exists delete
for index in pc.list_indexes().indexes:
    if index.name == new_index_name:
        pc.delete_index(new_index_name)

In [ ]:
from pinecone import PodSpec
pc.create_index(
    name=new_index_name, 
    dimension=dimension, 
    metric=metric, 
    spec=PodSpec(
        environment="gcp-starter",
        pod_type="starter",
        pods=1,
        replicas=1,
        shards=1
))

In [ ]:
pc.describe_index(new_index_name)

In [ ]:
index = pc.Index(name=new_index_name)

In [ ]:
index

In [ ]:
import random
ids = [ 'a', 'b', 'c', 'd']
vectors = [ [random.random() for _ in range(64)] for vec in range(4) ]

In [ ]:
ids

In [ ]:
len(vectors)

In [ ]:
# Bulk Insert some data
# [(id1, vecvtor1), (id2, vector2)]
index.upsert(vectors=zip(ids,vectors))

In [ ]:
index.describe_index_stats()

In [ ]:
# large amounts of data should be inserted in batches over serveral requests

In [ ]:
import itertools

vector_dimension = 64
vector_count = 1000

In [ ]:
# (id, vector)
example_data_generator = map (
    lambda i :
        (f'id-{i}', [random.random() for _ in range(vector_dimension)]), range(vector_count)
)

# helper function to split data into batches
def chunks(iterable, batch_size):
    
     it = iter(iterable)
     chunk = tuple(itertools.islice(it, batch_size))
     while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))
	

In [ ]:
for chunk in chunks(example_data_generator, 100):
    index.upsert(vectors=chunk)